# Generating incorrect answer suggestions
Using word embeddings we're going to find the most similar words to an answer.

## Importing the word embeddings
Unfortunately our beloved *spacy* does not offer most similar words. We'll use **gensim** for that.

Make sure you download the embeddings file first. Instructions in the *README* in the **data** folder.

In [1]:
import gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors

In [2]:
glove_file = '../data/embeddings/glove.6B.300d.txt'
tmp_file = '../data/embeddings/word2vec-glove.6B.300d.txt'

In [3]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_file, tmp_file)
model = KeyedVectors.load_word2vec_format(tmp_file)

## Similar words examples

In [4]:
model.most_similar(positive=['koala'], topn=10)

[('probo', 0.5426342487335205),
 ('koalas', 0.4729689359664917),
 ('orangutan', 0.4557289481163025),
 ('grizzly', 0.418164998292923),
 ('marsupial', 0.39361125230789185),
 ('wombat', 0.3832378685474396),
 ('cuddly', 0.3804110288619995),
 ('kodiak', 0.37843799591064453),
 ('kade', 0.37742379307746887),
 ('kangaroo', 0.3612629175186157)]

It seems to be working fine. Though what *the f* * is a probo?

![image.png](https://i.gyazo.com/8e982abd6da0025cb985b388c07507a8.png)

Ok.

At this point we asume that we have our answer, the sentence it's in, the entire text, and the title. Let's explore some words.

*__Oxygen__ is a chemical element with symbol O and atomic number 8.*  

In [5]:
model.most_similar(positive=['oxygen'], topn=10)

[('hydrogen', 0.63267982006073),
 ('nitrogen', 0.6251460313796997),
 ('helium', 0.5435217022895813),
 ('nutrients', 0.5369840860366821),
 ('breathing', 0.5023170709609985),
 ('chlorine', 0.4946938157081604),
 ('monoxide', 0.4911428689956665),
 ('dioxide', 0.4911195635795593),
 ('ammonia', 0.49079084396362305),
 ('carbon', 0.4836854338645935)]

That was easy. Let's try something more difficult.

*the oldest portuguese university was first established in **lisbon** before moving to coimbra.*

In [6]:
model.most_similar(positive=['lisbon'], topn=10)

[('portugal', 0.6408252716064453),
 ('porto', 0.5835250616073608),
 ('benfica', 0.5504174828529358),
 ('copenhagen', 0.5288482904434204),
 ('portuguese', 0.5266897678375244),
 ('madrid', 0.5219067335128784),
 ('brussels', 0.5173484683036804),
 ('oporto', 0.5147969126701355),
 ('prague', 0.5037161111831665),
 ('amsterdam', 0.5018222332000732)]

Seems like we are getting closer to *football teams* rather than *cities with old universities*. Let's add some more words from the sentence.

In [7]:
model.most_similar(positive=['lisbon', 'university'], topn=10)

[('faculty', 0.5288036465644836),
 ('college', 0.5237010717391968),
 ('professor', 0.5193325877189636),
 ('graduate', 0.5135289430618286),
 ('universities', 0.5098860263824463),
 ('copenhagen', 0.5022274255752563),
 ('campus', 0.494284987449646),
 ('prague', 0.4880773425102234),
 ('madrid', 0.4852182865142822),
 ('portugal', 0.4788099527359009)]

Great! But now the words are getting too close to university. It would be good if we can add more weight to the orignal answer.

I can manually do it by taking the best embeddings to the original answer and counting how many times they occur in the joint embeddings.

In [8]:
model.most_similar(positive=['lisbon', 'coimbra'], topn=10)

[('porto', 0.6089159250259399),
 ('portugal', 0.6070287823677063),
 ('oporto', 0.5988742113113403),
 ('braga', 0.5796492099761963),
 ('benfica', 0.5514551401138306),
 ('leiria', 0.5170067548751831),
 ('aveiro', 0.4983532130718231),
 ('viseu', 0.491713285446167),
 ('évora', 0.4914955198764801),
 ('são', 0.4868907928466797)]

Using another city really makes a difference and shows some good candidates. I think it'll be a good idea to use words in the sentence that are next to the answer.

### Words with the same stem

In [9]:
model.most_similar(positive=['write'], topn=10)

[('writing', 0.6969850063323975),
 ('read', 0.6291234493255615),
 ('wrote', 0.6251993179321289),
 ('written', 0.6065735816955566),
 ('publish', 0.56706303358078),
 ("'d", 0.5343195199966431),
 ('writes', 0.5341792702674866),
 ('tell', 0.5337096452713013),
 ('you', 0.5316604375839233),
 ('books', 0.5285096168518066)]

We could just remove all similar words that have the same stem as the original answer.

Additionally, the incorrect answers should be the same part of speech. Like with **write** - *read*, *publish*, *tell* are good candidates, but *books* could be easily discarded for being a noun.

### Numbers

In [10]:
model.most_similar(positive=['1944'], topn=10)

[('1943', 0.9581359624862671),
 ('1942', 0.9418259859085083),
 ('1941', 0.9256348609924316),
 ('1940', 0.8975383043289185),
 ('1945', 0.8817087411880493),
 ('1939', 0.8315709233283997),
 ('1946', 0.8234672546386719),
 ('1938', 0.781980574131012),
 ('1937', 0.7764101624488831),
 ('1935', 0.7516503930091858)]

Not that bad. They seem to gravitate around the events of WW2. It seems better than ramdon numbers or closest numbers if we need to have multiple answer question. But I think it may be a better question if you have to input the number yourself, and you get a better score if you are closer to the correct answer.

### Names

In [11]:
model.most_similar(positive=['bush'], topn=10)

[('clinton', 0.7889922857284546),
 ('obama', 0.7570987939834595),
 ('gore', 0.6871949434280396),
 ('w.', 0.6750580072402954),
 ('cheney', 0.6621243357658386),
 ('mccain', 0.6613168716430664),
 ('barack', 0.6568867564201355),
 ('administration', 0.6468127369880676),
 ('george', 0.6463572382926941),
 ('kerry', 0.6004412174224854)]

In [12]:
model.most_similar(positive=['euclid'], topn=10)

[('postulate', 0.4412063956260681),
 ('archimedes', 0.43941447138786316),
 ('n.e.', 0.39649105072021484),
 ('pythagoras', 0.39116498827934265),
 ('aristotle', 0.3895653486251831),
 ('avenue', 0.38695406913757324),
 ('proclus', 0.3855825960636139),
 ('greektown', 0.3836863040924072),
 ('ptolemy', 0.38028305768966675),
 ('berea', 0.37123364210128784)]

In [13]:
model.most_similar(positive=['atanasov'], topn=10)

[('atanas', 0.6365466713905334),
 ('fery', 0.44102150201797485),
 ('simeonov', 0.4386071562767029),
 ('atanassov', 0.4376071095466614),
 ('mladenov', 0.4347333014011383),
 ('sergeevich', 0.4314761757850647),
 ('neophytos', 0.4266960620880127),
 ('geleta', 0.41917920112609863),
 ('vassilev', 0.4189075827598572),
 ('stoev', 0.4143332839012146)]

I expected to be a lot worse. Names of famous people gets us other names of people with the same profesion - US presidents and greek mathematicians come up pretty easily. 

But with some less known figures, like a general in a certain battle, it woulnd't work. In those cases it would be good if we find other names in the same text or if we're working with a textbook we can use the names from other topics.

# Function

We'll keep it simple. We just need the *count* amount of distractors (incorrect answers).

In [14]:
def generate_distractors(answer, count):
    answer = str.lower(answer)
    
    ##Extracting closest words for the answer. 
    try:
        closestWords = model.most_similar(positive=[answer], topn=count)
    except:
        #In case the word is not in the vocabulary, or other problem not loading embeddings
        return []

    #Return count many distractors
    distractors = list(map(lambda x: x[0], closestWords))[0:count]
    
    return distractors

In [15]:
generate_distractors('oxygen', 4)

['hydrogen', 'nitrogen', 'helium', 'nutrients']

In [16]:
generate_distractors('bulgaria', 6)

['romania', 'hungary', 'ukraine', 'slovakia', 'bulgarian', 'macedonia']